In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u
import numpy as np
import tracemalloc
import time
import matplotlib.pyplot as plt
from scipy import special
import os.path

In [13]:
def run_planet(T_int,grav,a,T_star,logg,metal,rstar,mh,CtoO,rfacv):
    mhdeq=mh
    CtoOdeq=CtoO
    gases_fly = ['CO','CH4','H2O','NH3','CO2','N2','HCN','H2','PH3','C2H2','Na','K','TiO','VO','FeH']
    filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
    ck_db='/Users/sagnickmukherjee/Documents/GitHub/Disequilibrium-picaso/reference/climate_INPUTS/sonora_2020_feh'+mhdeq+'_co_'+CtoOdeq+'.data.196'
    
    opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)

    
    path = "/Users/sagnickmukherjee/Documents/research/geers/wasp96b"
    file_path= path+"/tpfile/tp_eq_planet_"+str(int(T_int))+"_grav_8.25_mh_"+mh+"_CO_"+CtoO+"_sm_0.0453_v_0.5_.dat"
    
    pressure,temp_guess = np.loadtxt(file_path,usecols=[0,1],unpack=True)
    file_temp = open(file_path)
    header = file_temp.readline()
    
    
    pressure =np.logspace(np.log10(1e-6),np.log10(200),91)
    
    # planet prop
    semi_major = a #0.0355 #"NA" # AU
    T_eff = T_int
    grav = grav
    #r_planet = 1 # needed for vulcan photochem runs

    ### star prop
    T_star = T_star
    logg = logg
    metal = metal
    r_star = rstar




    nlevel = 91
    
    kz = np.zeros(shape =(nlevel))+1e9
    nofczns = 1
    nstr1 = 80
    nstr = np.array([0,nstr1,89,0,0,0])
    rfacv = rfacv
    rfaci =1.0
    cloudy = False
    fsed = 2




    

    cl_run = jdi.inputs(calculation="planet", climate = True)

    cl_run.star(opacity_ck, temp =T_star,metal =metal, logg =logg, radius = r_star, radius_unit=u.R_sun,semi_major= semi_major , semi_major_unit = u.AU)#opacity db, pysynphot database, temp, metallicity, logg

    cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
    cl_run.T_eff(Teff=T_eff)





    cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mhdeq, CtoO = CtoO,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed, T_star = T_star, logg = logg, metal = metal, r_star = r_star, semi_major = semi_major)

    
    t0 = time.time()
    pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz,opd_,g0_,w0_ = cl_run.run_climate_model(opacity_ck,save_all_profiles=True,diseq_chem = True, self_consistent_kzz =False,vulcan_run = False, kz = kz, on_fly=True,gases_fly=gases_fly,mhdeq=mhdeq,CtoOdeq=CtoOdeq )
    t1 = time.time()
    print(t1-t0)
   
    
    
    return pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz,opd_,g0_,w0_

In [14]:
#T_int = np.array([100,200,300])
T_int=np.array([200])
grav = np.array([4.5])
T_star,logg,metal,rstar = 5326.6,4.38933,-0.03,0.939
mh = np.array(['+1.0'])
CtoO =np.array(['1.0'])

a,e = 0.0486, 0
cosv = np.array([0])
semi_major_ar = np.array([a])
rfacv = np.array([0.5])

for i in range(len(T_int)):
    for j in range(len(grav)):
        for k in range(len(mh)):
            for l in range(len(CtoO)):
                for m in range(len(semi_major_ar)):
                    for o in range(len(rfacv)):
                        pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz,opd_,g0_,w0_ = run_planet(T_int[i],grav[j],semi_major_ar[m],T_star,logg,metal,rstar,mh[k],CtoO[l],rfacv[o])


Iteration number  0 , min , max temp  837.5399696587097 4100.555361612217 , flux balance  467.19435149989573
Iteration number  1 , min , max temp  812.0197878656538 4039.1262215285738 , flux balance  56.05369812125068
Iteration number  2 , min , max temp  810.1379390844783 4031.493481045605 , flux balance  1.2694692683813964
Iteration number  3 , min , max temp  810.1299979415436 4031.000008304478 , flux balance  0.0010213852108446806
Iteration number  4 , min , max temp  810.1299951107682 4030.995378291079 , flux balance  1.9930703972681407e-07
Converged Solution in iterations  4
Big iteration is  810.1299951107682 0
Iteration number  0 , min , max temp  805.0763807955608 3927.5875640467907 , flux balance  76.5873646890064
Iteration number  1 , min , max temp  798.2557049975668 3818.5165875290318 , flux balance  3.1315402659307328
Iteration number  2 , min , max temp  798.1673874525451 3810.4648441814147 , flux balance  0.007672207671383046
Iteration number  3 , min , max temp  798.16

In [15]:
quench_levels=np.array([0,0,0,0])
path = "/Users/sagnickmukherjee/Documents/GitHub/picaso/Benchmarking/onfly/"

filename= path+"tpfile/tp_w39b_mh+1_co_1_kzz1e9.dat"
np.savetxt(filename, np.transpose([pressure,temp]))        

csv = path+"profile/profile_w39b_mh+1_co_1_kzz1e9.dat"
df.to_csv(csv,sep='\t')

